In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ2: In which phase of the technology life cycle path are the technologies?
# - Interest in topic: #Papers on a topic over time 
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
#- Publikationstyp: Workshop -> Conference -> Journal 

In [3]:
data = pd.read_csv('./data/data.csv',usecols = ['Publication Year','Article Citation Count','Author Keywords','Publisher'], dtype={'Author Keywords': str, 'Publisher': str})
data.head()

,Publication Year,Author Keywords,Article Citation Count,Publisher
0,2016,software engineering education;athletic softwa...,5.0,IEEE
1,2009,software engineering;knowledge management;lear...,28.0,IEEE
2,2016,global software engineering;agile software dev...,11.0,IEEE
3,2011,software engineering;computer and information ...,11.0,IEEE
4,2017,automotive software engineering;ASE;AUTOSAR;st...,4.0,IEEE


In [4]:
data.columns

Index(['Publication Year', 'Author Keywords', 'Article Citation Count',
       'Publisher'],
      dtype='object')

In [5]:
def IntrestInTopic(year):
    databyyear = data[data['Publication Year'] == year]
    

In [6]:
def FilterListErstellen(l):
    wordlist = []
    for x in l:
        wordlist.append(x[0])
    return pd.Series(wordlist,dtype=str)
    

In [7]:
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time

#df = data[['Authors','Cites']].groupby('Authors').sum()


#d = df['Cites'].loc[df['Cites'] > 400]
#d.plot.bar()
#print(df['Cites'])



result = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='Article Citation Count',
                        aggfunc=np.sum,
                        fill_value = 0)
result.head()

Publication Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Author Keywords,,,,,,,,,,,,,
- D.2.1Requirements/Specifications D.2.9.d Initiation and scope definition D.2.9 Management;Requirements/specifications;initiation and scope definition;management,0,0,0,0,0,0,0,12,0,0,0,0,0
11th International Symposium on Empirical Software Engineering and Measurement;13th International Conference on Predictive Models and Data Analytics in Software Engineering;21st International Systems and Software Product Line Conference;empirical software engineering;predictive models;data analytics;software product lines;product line engineering;developer productivity;issue resolution;agile development;microservices;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
33rd International Conference on Software Maintenance and Evolution;ICSME 17;32nd International Conference on Automated Software Engineering;ASE 17;software maintenance;software evolution;automated software engineering;flaky tests;technical debt;self-admitted technical debt;SATD;QA bots;regular expressions;regexes;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
40th International Conference on Software Engineering;17th International Conference on Software Reuse;gender;gender stereotypes;GenderMag;sentiment;sentiment analysis;Stack Overflow;Senti4SD;software product lines;SPL;software product line engineering;SPLE;variability management;safety-critical systems;software engineering education;software reuse;software development;software engineering;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,1,0,0,0
API Documentation;Code Sample;Usage Scenario;Stack Overflow;Crowd knowledge,0,0,0,0,0,0,0,0,0,0,3,0,0


In [8]:
words = {}
data = pd.DataFrame()
cnt = Counter()

for year in range(2009,2021,1):
    for keywords,count in result[year].items():
        for keyword in keywords.split(';'):
            cnt[keyword] += 1
            keys = list(words.keys())
            
            if keyword not in keys:
                words[keyword] = 0
            words[keyword] += count
             
    seriesyear = pd.Series(words).to_frame()
    seriesyear.columns = [year]
    data = data.merge(seriesyear, how= 'outer' , left_index = True, right_index = True)
    words = {}   
data    

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
- D.2.1Requirements/Specifications D.2.9.d Initiation and scope definition D.2.9 Management,0,0,0,0,0,0,0,12,0,0,0,0
Requirements/specifications,0,0,0,107,16,0,158,12,0,0,0,0
initiation and scope definition,0,0,0,0,0,0,46,12,0,0,0,0
management,0,147,0,37,86,0,10,12,7,0,5,0
11th International Symposium on Empirical Software Engineering and Measurement,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
service orchestration,0,0,0,0,0,1,0,0,0,0,0,0
Rundeck,0,0,0,0,0,1,0,0,0,0,0,0
working time of developers,0,0,0,0,0,0,0,0,0,0,0,0
IT company,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
anzahl_anzeigen = 25

In [10]:
cnt.most_common(anzahl_anzeigen)

[('software engineering', 8400),
 ('software development', 5184),
 ('software architecture', 972),
 ('software', 780),
 ('software testing', 588),
 ('requirements engineering', 492),
 ('software quality', 480),
 ('DevOps', 360),
 ('empirical software engineering', 336),
 ('agile development', 324),
 ('software maintenance', 324),
 ('empirical study', 324),
 ('refactoring', 324),
 ('machine learning', 324),
 ('requirements', 312),
 ('IEEE Software', 300),
 ('software design', 288),
 ('technical debt', 264),
 ('agile software development', 264),
 ('static analysis', 264),
 ('software requirements', 264),
 ('Software engineering', 252),
 ('Software testing', 252),
 ('software product lines', 240),
 ('reliability', 240)]

In [11]:
filterlist = FilterListErstellen(list(cnt.most_common(anzahl_anzeigen)))

In [12]:
filtered_data = data[data.index.isin(filterlist)]

In [13]:
filtered_data.head(anzahl_anzeigen)

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
empirical software engineering,164,5,0,50,80,22,7,8,0,17,3,12
software product lines,39,78,12,5,164,7,189,2,0,1,0,0
agile development,0,47,0,57,0,0,0,12,13,20,0,0
software engineering,587,1226,394,433,410,563,1346,1771,1060,434,13,9
software development,113,45,100,42,23,58,633,1755,988,415,7,0
software maintenance,0,115,0,86,59,18,31,92,0,1,6,0
technical debt,0,0,20,333,0,0,26,26,1,1,8,1
empirical study,0,0,0,0,265,117,31,34,83,27,10,20
software quality,95,103,67,312,11,101,225,45,19,2,11,2
software testing,46,0,15,55,34,0,180,335,72,7,2,34


In [14]:
#- Publikationstyp: Workshop -> Conference -> Journal 
#data['Author Keywords'] = data['Author Keywords'].str.split(';',expand=False)

#data['Author Keywords'].head()

#data[data['Author Keywords'].str.split(';',expand=False).Contains]
#Mit isin Liste abfragen
#Erst Topic Filtern und dann weiter Arbeiten

#test = {}

#words = Counter()
#data[data['Publication Year'] == year]
#for year in range(2009,2021,1):
#    for columns in data[data['Publication Year'] == year].itertuples():

#        text = columns[2] #
#        if type(text) is str:
#            for topic in text.split(';'):
#                words[topic] += 1

#        test[year] = list(words)

    #print(words.most_common(40))   

In [15]:
#print(test[2019])

In [16]:
#import spacy
#from spacy.symbols import NOUN


#nlp = spacy.load("en_core_web_sm")

#cnt = Counter()
#tokenlist = []

#for columns in data.itertuples():

#    text = columns[2] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
#    if type(text) is str:
#        doc = nlp(text)
        
#        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
#            token_text = token.text
#            token_pos = token.pos_
#            token_dep = token.dep_
#            token_ent = token.ent_type_
            
#            if token.pos == NOUN:
#                if not token.is_stop:
#                    #print(f"{token_text} | {token_ent} | {token.tag_}")
#                    cnt[token_text] += 1
#                    tokenlist.append([token_text,columns[2],token_ent])
